In [1]:
import plotly.express as px
import pandas as pd
import pickle

In [50]:
df = pickle.load(open("data/loan_sum_by_state.pkd", "rb"))
df = df.drop(labels=45, axis=0)
df = df.drop(labels=['a', 'b', 'c', 'd', 'e'], axis=1)
df = df.sort_index()

### Map for state abbreviation to name

In [15]:
abbr_map = pd.read_csv(open("data/state_abbr_latlng.csv", "rb"), header=None)
d = {0:'abbr', 1:'lat', 2:'long', 3:'state'}
abbr_map = abbr_map.rename(columns=d)
abbr_map = abbr_map.drop(columns=['lat', 'long'])

In [112]:
abbr_mapping = list(zip(abbr_map['abbr'].unique(), abbr_map['state'].unique()))

In [114]:
abbr_dict = {}
for k, v in abbr_mapping:
    abbr_dict[k] = v

In [77]:
unem = pd.read_csv(open("data/nst-est2019-popchg2010_2019.csv", "rb"), usecols=['NAME', 'POPESTIMATE2019'])

In [120]:
df['Name'] = df.apply(lambda row: abbr_dict[row['State']], axis=1)

In [110]:
unem = unem.rename(columns={'NAME':'State', 'POPESTIMATE2019':'pop'})

In [123]:
unem = unem.rename(columns={'State':'Name'})

In [132]:
joined = unem.set_index('Name').join(df.set_index('Name')).reset_index()

In [136]:
loan_sum_by_pop = joined.loc[:, ['Name', 'pop', 'loan_sum']]

In [138]:
pickle.dump(loan_sum_by_pop, open("data/loan_sum_by_pop.pkd", "wb"))

In [143]:
loan_sum_by_pop.rename(columns={'loan_sum':'sum'})

,Name,pop,sum
0,Alabama,4903185,4801.975
1,Alaska,731545,968.725
2,Arizona,7278717,6587.450
3,Arkansas,3017804,2262.650
4,California,39512223,52530.100
5,Colorado,5758736,7670.600
6,Connecticut,3565287,5088.950
7,Delaware,973764,1159.225
8,Florida,21477737,22803.775
9,Georgia,10617423,10839.900


In [147]:
loan_sum_by_pop['percap'] = loan_sum_by_pop.apply(lambda row: row['loan_sum']/row['pop'], axis=1)

In [158]:
loan_sum_by_pop = loan_sum_by_pop.sort_values(by=['percap'])

In [159]:
pickle.dump(loan_sum_by_pop, open("data/loan_dollars_per_cap.pkd", "wb"))

In [151]:
loan_sum_by_pop.columns

Index(['Name', 'pop', 'loan_sum', 'percap'], dtype='object')

In [160]:
fig = px.bar(loan_sum_by_pop, x="Name", y="percap", labels={'x':'State', 'y':'Loan Dollars/Resident ($)'},
                  title="Loan Dollars per Capita by State")

In [162]:
fig.update_xaxes(title_text='State')
fig.update_yaxes(title_text='Loan Dollars/Resident ($)')
fig.show()

In [166]:
fig.write_html("templates/loan_sum_per_cap.html", include_plotlyjs="cdn")

In [165]:
fig.write_image("loan_sum_per_cap.png")